In [11]:
import requests
import json
import pandas as pd
from time import sleep

In [12]:
df = pd.read_csv('../output/codici_umbria_er_20241117.csv', dtype=str)
df_comuni = df[df['tipo'] == 'CM']
# Filtering for only Umbria
df_er = df_comuni[df_comuni['cod_regione'] == '10']

In [13]:
BASE_URL = 'https://eleapi.interno.gov.it/siel/PX/getprefeR/DE/20241117/TE/07/RE/'

df_er['url_aff'] = df_er.apply(
    lambda row: f"{BASE_URL}{row['cod_regione']}/PR/{row['cod_provincia_2']}/CM/{row['cod_comune']}", axis=1
)

df_er['url'] = df_er.apply(
    lambda row: f"https://elezioni.interno.gov.it/risultati/20241117/regionali/scrutini/italia/{row['cod_regione']}{row['cod_provincia']}{row['cod_comune']}", axis=1
)
df_er.sample(3)

/var/folders/m0/g7ky_r7x67l1pv2zgg_fptx80000gn/T/ipykernel_93918/3249733948.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_er['url_aff'] = df_er.apply(
/var/folders/m0/g7ky_r7x67l1pv2zgg_fptx80000gn/T/ipykernel_93918/3249733948.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_er['url'] = df_er.apply(


,cod,desc,tipo,tipo_comune,dt_agg,cod_ele,cod_regione,regione,cod_provincia,provincia,cod_provincia_2,cod_comune,url_aff,url
420,103040800080,CALVI DELL'UMBRIA,CM,N,NaN,7,10,UMBRIA,304,TERNI,080,0080,https://eleapi.interno.gov.it/siel/PX/getprefe...,https://elezioni.interno.gov.it/risultati/2024...
403,103040580520,TODI,CM,M,NaN,7,10,UMBRIA,304,TERNI,058,0520,https://eleapi.interno.gov.it/siel/PX/getprefe...,https://elezioni.interno.gov.it/risultati/2024...
377,103040580260,MAGIONE,CM,N,NaN,7,10,UMBRIA,304,TERNI,058,0260,https://eleapi.interno.gov.it/siel/PX/getprefe...,https://elezioni.interno.gov.it/risultati/2024...


In [14]:
headers = {
    'accept': 'application/json, text/plain, */*',
    'accept-language': 'en-US,en;q=0.6',
    'dnt': '1',
    'origin': 'https://elezioni.interno.gov.it',
    'priority': 'u=1, i',
    'referer': 'https://elezioni.interno.gov.it/',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-site',
    'sec-gpc': '1'
}

In [ ]:
comuni_data = []

for idx, url in enumerate(df_er['url_aff']):
    print(f"\nProcessing record {idx + 1}")
    print(f"URL: {url}")

    response = requests.get(url, headers=headers)
    print(f"Response status code: {response.status_code}")
    
    data = response.json()

    comune = df_er.iloc[idx]['desc']
    provincia = df_er.iloc[idx]['provincia']
    regione = df_er.iloc[idx]['regione']
    url = df_er.iloc[idx]['url']

    print(f"Processing data for Comune: {comune}, Provincia: {provincia}")

    # Start with basic comune info
    comune_info = {
        'comune': comune,
        'provincia': provincia,
        'regione': regione,
        'url': url
    }
    
    # Add votes for each lista using lista description as column name
    if 'liste' in data:
        for lista in data['liste']:
            lista_desc = lista['desc']
            voti = lista.get('voti', 0)  # Use 0 if no votes found
            comune_info[lista_desc] = voti
            print(f"{lista_desc}: {voti} votes")

    sezioni_scrutinate = data['liste'][0]['sez_perv']
    sezioni_tot = data['liste'][0]['sez_tot']

    # Optional: Verify consistency across lists
    if not all(lista.get('sez_perv') == sezioni_scrutinate for lista in data['liste']):
        print(f"Warning: Inconsistent scrutinized sections for {comune}")

    # Add to comune_info
    comune_info['sezioni_scrutinate'] = sezioni_scrutinate
    comune_info['sezioni_tot'] = sezioni_tot

    comuni_data.append(comune_info)
    
    print(f"Processed {comune}")
    sleep(0.1)

# Create DataFrame
print("\nCreating DataFrame...")
df_data = pd.DataFrame(comuni_data)

print("\nSample of processed data:")
print(df_data.head())


Processing record 1
URL: https://eleapi.interno.gov.it/siel/PX/getprefeR/DE/20241117/TE/07/RE/10/PR/058/CM/0010


KeyboardInterrupt: 

In [ ]:
df_data['comune'] = df_data['comune'].str.title()
df_data['provincia'] = df_data['provincia'].str.title()
df_data['regione'] = df_data['regione'].str.title()
df_data

,comune,provincia,regione,url,FRONTE DEL DISSENSO,QUINTO POLO PER L'ITALIA,INSIEME PER UN'UMBRIA RESISTENTE,PIU' ITALIA SOVRANA,LEGA,ALTERNATIVA POPOLARE,...,ALLEANZA VERDI E SINISTRA,CIVICI UMBRI,ALTERNATIVA PER L'UMBRIA,FORZA DEL POPOLO,ALTERNATIVA RIFORMISTA RIZZO PRESIDENTE,TRITTO PRESIDENTE UMANI INSIEME LIBERI,UMBRIA DOMANI PROIETTI PRESIDENTE,UMBRIA FUTURA - RIFORMISTI E CIVICI PROIETTI PRESIDENTE,sezioni_scrutinate,sezioni_tot
0,Assisi,Terni,Umbria,https://elezioni.interno.gov.it/risultati/2024...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,32
1,Bastia Umbra,Terni,Umbria,https://elezioni.interno.gov.it/risultati/2024...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,22
2,Bettona,Terni,Umbria,https://elezioni.interno.gov.it/risultati/2024...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
3,Bevagna,Terni,Umbria,https://elezioni.interno.gov.it/risultati/2024...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,6
4,Campello Sul Clitunno,Terni,Umbria,https://elezioni.interno.gov.it/risultati/2024...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,Porano,Terni,Umbria,https://elezioni.interno.gov.it/risultati/2024...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
88,San Gemini,Terni,Umbria,https://elezioni.interno.gov.it/risultati/2024...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5
89,San Venanzo,Terni,Umbria,https://elezioni.interno.gov.it/risultati/2024...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,7
90,Stroncone,Terni,Umbria,https://elezioni.interno.gov.it/risultati/2024...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,6


In [ ]:
df_data.to_csv('../output/risultati_um.csv', index=False, encoding='UTF-8-sig')